In [8]:
#------------------------------------------- bibliotecas

import boto3 # conexão com AWS
import zipfile # trabalhar com arquivoz .zip
from io import BytesIO # armazenar o conteúdo do objeto

In [ ]:
#------------------------------------------- configurações AWS

s3 = boto3.client('s3') # instância para conectar à AWS

# configurações do bucket
bucket = 'elasticbeanstalk-us-east-2-047710767346'

# especificando e criando os prefixos (equivalentes a "subpastas")
prefixo_mae = 'Case_A3_refatorado'
s3.put_object(Bucket=bucket, Key=f'{prefixo_mae}/')

# origem
prefixo_raw = f'{prefixo_mae}/raw_arquivos_zip'

# destino
prefixo_bronze = f'{prefixo_mae}/bronze_arquivos_extraidos'

In [3]:
#------------------------------------------- definindo e listando objetos a serem extraídos

obj = s3.list_objects(Bucket=bucket, Prefix=f'{prefixo_raw}/Microdados')

# obj será um dicionário ( {'chave':'valor'}) em que as keys (nome) dos arquivos zip estão armazenadas na chave 'contents';
# precisamos acessar a chave 'contents', que será uma lista de dicionários com as infos dos zips;
# aí vamos querer a chave 'key', que contém o nome do arquivo

arquivos_zip = obj['Contents'] # arquivos_zip será uma lista com as infos e keys de cada objeto

In [22]:
#------------------------------------------- loop para percorrer os objetos listados e extrair, escrevendo o conteúdo na bronze

for zip_info in arquivos_zip:
    zip_key = zip_info['Key']
    nome_zip = zip_key.split('/')[-1]

    # verificar se o arquivo é um ZIP
    if not nome_zip.endswith('.zip'):
        raise ValueError(f'O arquivo {nome_zip} não é um arquivo ZIP válido.')

    # requisição para obter o conteúdo do ZIP
    requisicao_conteudo_zip = s3.get_object(Bucket=bucket, Key=zip_key)
    conteudo_zip = BytesIO(requisicao_conteudo_zip['Body'].read())

    # extração e carregamento direto para o S3
    with zipfile.ZipFile(conteudo_zip, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            with zip_ref.open(file_info.filename) as file:
                file_content = file.read()

            # escrever o conteúdo extraído diretamente para o S3
            s3.put_object(Body=file_content, Bucket=bucket, Key=f'{prefixo_bronze}/{file_info.filename}')

    print(f'O conteúdo do ZIP {nome_zip} foi extraído para: {prefixo_bronze}')

print('Extração finalizada.')


O conteúdo do ZIP Microdados_1995.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_1996.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_1997.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_1998.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_1999.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_2000.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_2001.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_2002.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_2003.zip foi extraído para: Case_A3_refatorado/bronze_arquivos_extraidos
O conteúdo do ZIP Microdados_2004.zip foi extraído para: Case_A3_refatorado/bronze